In [7]:
from decouple import Config, RepositoryEnv
from json import dumps
import json
import openai
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
import pandas as pd
from pydantic import BaseModel, ValidationError, Field
from typing import List, Tuple , Union, Optional
from langchain.schema import HumanMessage, SystemMessage, AIMessage

In [8]:
from sentence_transformers import SentenceTransformer, util

In [9]:
import time
import matplotlib.pyplot as plt
import seaborn as sns

In [10]:
from openai import OpenAI

In [11]:
DOTENV_FILE = 'C:\\Users\\Tiziano Pacifico\\Desktop\\OpenAICourses\\.env'
env_config = Config(RepositoryEnv(DOTENV_FILE))
openai.api_key = env_config.get('CHATGPT_API_KEY')

In [15]:
path = "C:\\Users\\Tiziano Pacifico\\Desktop\\TESI\\RedditDS\\chatGPT\\BERTopic_out\\"
filename = "BERTopic_out_df.json"
df = pd.read_json(path+filename,lines=True)

In [16]:
filename = "BERTopic_id_list.json"
df_id = pd.read_json(path+filename,lines=True)

In [17]:
df_final = pd.merge(df, df_id, on='Topic')

In [20]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73943 entries, 0 to 73942
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Document  73943 non-null  object
 1   Topic     73943 non-null  int64 
 2   id        73943 non-null  object
 3   BERT_out  73943 non-null  object
dtypes: int64(1), object(3)
memory usage: 2.3+ MB


In [21]:
def create_topwords_string(top_words_list):
    topwords_string = ''
    for line in top_words_list:
        topwords_string += f"({line[0]}, {line[1]}) "
    return topwords_string

In [22]:
df = df_final

In [23]:
df['top_words_str'] = df['BERT_out'].apply(create_topwords_string)

In [24]:
df.head()

,Document,Topic,id,BERT_out,top_words_str
0,chatgpt stand dont think michael anyth hey ple...,7,134stgc,"[[bot, 0.12105691510000001], [pleas, 0.0709474...","(bot, 0.12105691510000001) (pleas, 0.070947460..."
1,chatgpt app give free chat pleas respond comme...,7,134rp29,"[[bot, 0.12105691510000001], [pleas, 0.0709474...","(bot, 0.12105691510000001) (pleas, 0.070947460..."
2,automat grammar visual model chatgpt pleas res...,7,134rk8e,"[[bot, 0.12105691510000001], [pleas, 0.0709474...","(bot, 0.12105691510000001) (pleas, 0.070947460..."
3,host pleas respond comment prompt use output p...,7,134r7kv,"[[bot, 0.12105691510000001], [pleas, 0.0709474...","(bot, 0.12105691510000001) (pleas, 0.070947460..."
4,there pleas respond comment prompt use output ...,7,134r49c,"[[bot, 0.12105691510000001], [pleas, 0.0709474...","(bot, 0.12105691510000001) (pleas, 0.070947460..."


In [25]:
system_message = """You will act as a topic label creator. \
You will create a label that identify a topic given a set of 30 keywords that are meaningfull for the topic.\
                               The topic can be one word or one short sentence, depending how many words you need\
                              to capture the essence of the text. The text will be given in input as human message. Give as output just the sentence or the words that identify the topic.\
                              The keywords list will be given as a result of a BERTopic alghoritm over a short article in the following format: ###\
                              \(celebrities, 0.0569693501) (influence, 0.0549841217) (their, 0.0467051602) (the, 0.044765043500000004) (and, 0.0443447511) (culture, 0.042227265) (to, 0.0411800998)....\
                                where each number after the word state how important is the word in the topic description in terms of c-TF_IDF score"""

In [26]:
BERT_out = list(df['top_words_str'].unique())

In [27]:
GPT_topics = []
client = OpenAI()
for i,out in enumerate(BERT_out):
    user_message = f"\nCreate a topic label for the following BERTopic output{out}"
    topic = client.chat.completions.create(
        model="gpt-4-1106-preview",
        messages= [
            {"role":"system", "content":system_message},
            {"role":"user", "content":user_message}
        ]
    )
    print(f"elaborato messaggio n: {i}")
    GPT_topics.append((out,topic))

elaborato messaggio n: 0
elaborato messaggio n: 1
elaborato messaggio n: 2
elaborato messaggio n: 3
elaborato messaggio n: 4
elaborato messaggio n: 5
elaborato messaggio n: 6
elaborato messaggio n: 7
elaborato messaggio n: 8
elaborato messaggio n: 9
elaborato messaggio n: 10
elaborato messaggio n: 11
elaborato messaggio n: 12
elaborato messaggio n: 13
elaborato messaggio n: 14
elaborato messaggio n: 15
elaborato messaggio n: 16
elaborato messaggio n: 17
elaborato messaggio n: 18
elaborato messaggio n: 19
elaborato messaggio n: 20
elaborato messaggio n: 21


In [28]:
GPT_topics[1][1].choices[0].message.content

'Chatbot Interaction and Usage on Discord'

In [29]:
GPT_topics_str = []
for s in GPT_topics:
    GPT_topics_str.append((s[0],s[1].choices[0].message.content))

In [30]:
GPT_topics_clean = []
for topic in GPT_topics_str:
    if len(topic[1].split("\"")) > 1:
        GPT_topics_clean.append((topic[0],topic[1].split("\"")[1]))
    else:GPT_topics_clean.append(topic)

In [31]:
def add_topic_label(df_col):
    for topic in GPT_topics_clean:
        if topic[0] == df_col:
            return topic[1]

In [32]:
df['GPT_label'] = df['top_words_str'].apply(add_topic_label)

In [33]:
df.head()

,Document,Topic,id,BERT_out,top_words_str,GPT_label
0,chatgpt stand dont think michael anyth hey ple...,7,134stgc,"[[bot, 0.12105691510000001], [pleas, 0.0709474...","(bot, 0.12105691510000001) (pleas, 0.070947460...",AI Chatbot Interaction and Assistance on Cloud...
1,chatgpt app give free chat pleas respond comme...,7,134rp29,"[[bot, 0.12105691510000001], [pleas, 0.0709474...","(bot, 0.12105691510000001) (pleas, 0.070947460...",AI Chatbot Interaction and Assistance on Cloud...
2,automat grammar visual model chatgpt pleas res...,7,134rk8e,"[[bot, 0.12105691510000001], [pleas, 0.0709474...","(bot, 0.12105691510000001) (pleas, 0.070947460...",AI Chatbot Interaction and Assistance on Cloud...
3,host pleas respond comment prompt use output p...,7,134r7kv,"[[bot, 0.12105691510000001], [pleas, 0.0709474...","(bot, 0.12105691510000001) (pleas, 0.070947460...",AI Chatbot Interaction and Assistance on Cloud...
4,there pleas respond comment prompt use output ...,7,134r49c,"[[bot, 0.12105691510000001], [pleas, 0.0709474...","(bot, 0.12105691510000001) (pleas, 0.070947460...",AI Chatbot Interaction and Assistance on Cloud...


In [34]:
filename = "BERTopic_gpt_out_df.json"
df.to_json(path+filename, orient="records", lines=True)

In [35]:
df['GPT_label'].unique()

array(['AI Chatbot Interaction and Assistance on Cloud Platforms',
       'Chatbot Interaction and Usage on Discord',
       'Dan Discussing ChatGPT Jailbreaking Ethics',
       'AI and ChatGPT in Everyday Life',
       'Reddit Karma and Account Restrictions',
       'Social Media Karma and Automated Account Moderation',
       'ChatGPT Use Cases and Discussions',
       'Obama and Trump References in AI Conversations',
       'AI Chatbot Interaction and Response automation',
       'Social Media Automated Responses and Bots Interaction',
       'AI Bots Managing Online Interactions in Chat Platforms',
       'NFT Collecting and Discord Community Interaction',
       "Bing's AI Chatbot and User Interaction",
       'Discord Server Chatbot Interaction and User Requests',
       'Spam Threshold Management in Online Communities',
       'Discord Moderation Bot Commands and User Interaction',
       'Bot Moderation in Online Comment Sections',
       'Spam Prevention and Account Karma Mana

In [36]:
df_id_gpt = df[['Topic','GPT_label']].sort_values(by='Topic').drop_duplicates(subset='Topic').reset_index(drop=True)

In [37]:
df_id_gpt

,Topic,GPT_label
0,0,AI and ChatGPT in Everyday Life
1,1,Chatbot Interaction and Usage on Discord
2,2,Discord Server Chatbot Interaction and User Re...
3,3,AI Chatbot Interaction and Response automation
4,4,Bot Moderation in Online Comment Sections
5,5,AI Bots Managing Online Interactions in Chat P...
6,6,Discord Server Moderation and ChatGPT Interaction
7,7,AI Chatbot Interaction and Assistance on Cloud...
8,8,Social Media Automated Responses and Bots Inte...
9,9,Social Media Karma and Automated Account Moder...
